In [257]:
'''Small Parsimony Problem. Find the most parsimonious labeling of the internal nodes of a rooted tree.
     Input: A rooted binary tree with each leaf labeled by a string of length m.
     Output: A labeling of all other nodes of the tree by strings of length m that minimizes the parsimony score of the tree.'''


'''SmallParsimony(T, Character)
    for each node v in tree T
        Tag(v) ← 0
        if v is a leaf
            Tag(v) ← 1
            for each symbol k in the alphabet
                if Character(v) = k
                    sk(v) ← 0
                else
                    sk(v) ← ∞
    while there exist ripe nodes in T
        v ← a ripe node in T
        Tag(v) ← 1
        for each symbol k in the alphabet
            sk(v) ← minimumall symbols i {si(Daughter(v))+δi,k} + minimumall symbols j {sj(Son(v))+δj,k}
    return minimum over all symbols k {sk(v)}'''

def delta(i,k):
    if i==k:
        return 0
    else:
        return 1

def SmallParsimony_rooted(T,seqs):
    '''Small Parsimony Problem. Find the most parsimonious labeling of the internal nodes of a rooted tree.
     Input: A rooted binary tree with each leaf labeled by a string of length m.
     T={0:[4],1:[4],2:[5],3:[5],4:[0,1,6],5:[2,3,6],6:[4,5]}
     seqs={0:'CAAATCCC',1:'ATTGCGAC',2:'CTGCGCTG',3:'ATGGACGA',4:'',5:'',6:''}
     Output: A labeling of all other nodes of the tree by strings of length m that minimizes the parsimony score of the tree
     tree_score=16
     Character={0: 'CAAATCCC', 1: 'ATTGCGAC', 2: 'CTGCGCTG', 3: 'ATGGACGA', 4: 'ATAGCCAC', 5: 'ATGGACTA', 6: 'ATAGACAA'}
     '''
    #from Bioinf_functions import delta
    ### find sequence length ###
    for k,v in seqs.items():
        if len(v)>0:
            sequence_len=len(v)
            break
    ### Init sequence dict, and tree score
    Character=seqs.copy()
    Character = Character.fromkeys(Character, '')
    tree_score=0
    ### proceed nucleotibe by nucleotide ###
    for n in range(sequence_len):
        
        leaves=[k for k,v in T.items() if len(T[k])==1]
        Tag={} #marks which nodes were visited
        s={'A':{},'C':{},'G':{},'T':{}}
        minscore_nucl={}
        #### initiate ####
        for v in T.keys():
            Tag[v]=0
            if v in leaves:
                Tag[v]=1 #mark node as visited
                for k in ['A','C','G','T']:
                    if k==seqs[v][n]:
                        s[k].update({v:0})
                        minscore_nucl[v]=k
                    else:
                        s[k].update({v:float('Inf')})
        #### calculate parsimony score  ####
        ripe_nodes=[k for k,v in Tag.items() if v==0] #find nodes not visited yet
        while len(ripe_nodes)>0:
            for v in ripe_nodes:
                Tag[v]=1
                for k in ['A','C','G','T']:
                    s[k].update({v:sum([min([s[i][item]+delta(i,k) for i in s.keys() ]) for item in T[v] if item<=v])})
                minscore=min([s[k][v] for k in s.keys()])
                minscore_nucl[v]=[key for key,val in s.items() if s[key][v]==minscore ]
            ripe_nodes=[k for k,v in Tag.items() if v==0]
        #### backtrack ####
        root=max(T.keys()) #last common ancester/root
        minval=min([s[k][root] for k in s.keys()]) #lowest score for last common ancestor
        for key in s.keys():
            if s[key][root] ==minval:
                Ancestral_char=key # Ancestral character
                break
        tree_score+=minval
        Character[root] += Ancestral_char
        queue = []
        queue.append(root)
        while queue:
            parent_node = queue.pop()
            son = T[parent_node][0]
            daughter = T[parent_node][1]
            parent_Char = Character[parent_node][n]
            if parent_Char in minscore_nucl[son]:
                Character[son]+=parent_Char
            else: 
                Character[son]+=minscore_nucl[son][0]
            if parent_Char in minscore_nucl[daughter]:
                Character[daughter]+=parent_Char
            else: 
                Character[daughter]+=minscore_nucl[daughter][0]
            if len(T[son])>1:
                queue.append(son)
            if len(T[daughter])>1:
                queue.append(daughter)
    return tree_score,Character

### tests ###

#T={0:[8],1:[8],2:[9],3:[9],4:[10],5:[10],6:[11],7:[11],8:[0,1,12],9:[2,3,12],10:[4,5,13],11:[6,7,13],12:[8,9,14],13:[10,11,14],14:[12,13]}
#seqs={0:'C',1:'C',2:'A',3:'C',4:'G',5:'G',6:'T',7:'C',8:'',9:'',10:'',11:'',12:'',13:'',14:''}


#T={0:[7],1:[7],2:[8],3:[8],4:[11],5:[9],6:[9],7:[0,1,10],8:[2,3,10],9:[6,5,11],10:[7,8],11:[9,4],12:[10,11]}
#seqs={0:'C',1:'C',2:'A',3:'C',4:'G',5:'T',6:'C',7:'',8:'',9:'',10:'',11:'',12:''}

seqs={0:'CAAATCCC',1:'ATTGCGAC',2:'CTGCGCTG',3:'ATGGACGA',4:'',5:'',6:''}
T={0:[4],1:[4],2:[5],3:[5],4:[0,1,6],5:[2,3,6],6:[4,5]} 

'''
#f=open("input2.txt",'r')
#f=open("input.txt",'r')
#f=open("../../Downloads/dataset_10335_10 (1).txt",'r')
f=open("../../Downloads/rosalind_ba7f (1).txt",'r')
lines=f.read().splitlines()
f.close()
No_leaves=lines[0]
adj=lines[1:]
#print adj

T={}
n=0
seqs={}
for item in adj:
    node=item.split('->')
    node[0]=int(node[0])
    try:
        node[1]=int(node[1])
        if node[0] in T.keys():
            T[node[0]]+=[node[1]]
        else:
            T[node[0]]=[node[1]]
        T[node[1]]+=[node[0]]
        seqs[node[0]]=''
    except ValueError:
        if node[0] in T.keys():
            T[node[0]]+=[n]
        else:
            T[node[0]]=[n]
        T[n]=[node[0]]
        seqs[n]=node[1]
        sequence_len=len(node[1])
        seqs[node[0]]=''
    n+=1
'''    
score,sequence=SmallParsimony_rooted(T,seqs)
print score,sequence


fa=open('output.txt','w')
entry=str(score)
print entry
fa.write(entry+'\n')
fa.close()

        
from Bioinf_functions import HammingDistance
tree=T.copy()
for key,value in tree.items():
    for item in value:
        fa=open('output.txt','a')
        entry=str(sequence[key])+'->'+str(sequence[item])+':'+str(HammingDistance(sequence[key], sequence[item]))
        #print entry
        fa.write(entry+'\n')
        fa.close()
   

16 {0: 'CAAATCCC', 1: 'ATTGCGAC', 2: 'CTGCGCTG', 3: 'ATGGACGA', 4: 'ATAGCCAC', 5: 'ATGGACTA', 6: 'ATAGACAA'}
16


In [269]:
'''Small Parsimony Problem in an Unrooted Tree Problem: Find the most parsimonious labeling of the internal nodes in an unrooted tree.
     Input: An unrooted binary tree with each leaf labeled by a string of length m.
     Output: A labeling of all other nodes of the tree by strings of length m that minimizes the parsimony score of the tree.'''


def SmallParsimony_unrooted(T,seqs):
    '''
    INPUT
    T =  {0: [4], 1: [4], 2: [5], 3: [5], 4: [0, 1, 5], 5: [2, 3, 4]}
    seqs =  {0: 'TCGGCCAA', 1: 'CCTGGCTG', 2: 'CACAGGAT', 3: 'TGAGTACC', 4: '', 5: ''}
    OUTPUT
    score=17
    sequence={0: 'TCGGCCAA', 1: 'CCTGGCTG', 2: 'CACAGGAT', 3: 'TGAGTACC', 4: 'CCTGGCAA', 5: 'CAAGGAAC'}
    '''
    #from Bioinf_functions import delta,SmallParsimony_rooted
    #### find two internal nodes
    for k,v in T.items():
        if len(v)>1:
            node0=k
            for item in v:
                if len(T[item])>1:
                    node1=item
                    break
            
    #### turn unrooted tree to rooted tree by adding root between the two internal nodes
    new_node=len(T)
    node0_newlinks=[item for item in T[node0] if item!= node1]+[new_node]
    node1_newlinks=[item for item in T[node1] if item!= node0]+[new_node]

    Tree=T.copy()
    seqs.update({new_node:''})
    Tree.update({node0:node0_newlinks,node1:node1_newlinks,new_node:[node0,node1]})

    #### run SmallParsimony

    score,sequence=SmallParsimony_rooted(Tree,seqs)
    sequence.pop(new_node)
    return score,sequence

### tests ###

seqs={0: 'TCGGCCAA', 1: 'CCTGGCTG', 2: 'CACAGGAT', 3: 'TGAGTACC', 4: '', 5: ''}
T={0: [4], 1: [4], 2: [5], 3: [5], 4: [0, 1, 4], 5: [2, 3, 4]}



f=open("input.txt",'r')
#f=open("input2.txt",'r')
#f=open("../../Downloads/dataset_10335_12.txt",'r')
#f=open("../../Downloads/rosalind_ba7g.txt",'r')
lines=f.read().splitlines()
f.close()
No_leaves=lines[0]
adj=lines[1:]

T={}
n=0
seqs={}
for item in adj:
    node=item.split('->')
    try:
        node[0]=int(node[0]) #first item is a number(4->AATGGC)
        try:
            node[1]=int(node[1])
            node0=node[0] #record internal node 
            node1=node[1] #record internal node (the root will be placed between node0 and node1)
            if node[0] in T.keys():
                T[node[0]]+=[node[1]]
            else:
                T[node[0]]=[node[1]]
                seqs[node[0]]=''
        except ValueError:    
            if node[0] in T.keys():
                T[node[0]]+=[n]
                T[n]=[node[0]]
                seqs[n]=node[1]
            else:
                T[node[0]]=[n]
                T[n]=[node[0]]
                seqs[n]=node[1]
                sequence_len=len(node[1])
                seqs[node[0]]=''
            n+=1
    except ValueError:
        pass

print 'T = ',T 
print 'seqs = ',seqs

score,sequence=SmallParsimony_unrooted(T,seqs)
print score,sequence

#### Format output using T (unrooted)not Tree (rooted).

fa=open('output.txt','w')
entry=str(score)
print entry
fa.write(entry+'\n')
fa.close()

        
from Bioinf_functions import HammingDistance
tree=T.copy()
#print tree
for key,value in tree.items():
    for item in value:
        fa=open('output.txt','a')
        entry=str(sequence[key])+'->'+str(sequence[item])+':'+str(HammingDistance(sequence[key], sequence[item]))
        print entry
        fa.write(entry+'\n')
        fa.close()
  

T =  {0: [4], 1: [4], 2: [5], 3: [5], 4: [0, 1, 5], 5: [2, 3, 4]}
seqs =  {0: 'TCGGCCAA', 1: 'CCTGGCTG', 2: 'CACAGGAT', 3: 'TGAGTACC', 4: '', 5: ''}
17 {0: 'TCGGCCAA', 1: 'CCTGGCTG', 2: 'CACAGGAT', 3: 'TGAGTACC', 4: 'CCTGGCAA', 5: 'CAAGGAAC'}
17
TCGGCCAA->CCTGGCAA:3
CCTGGCTG->CCTGGCAA:2
CACAGGAT->CAAGGAAC:4
TGAGTACC->CAAGGAAC:4
CCTGGCAA->TCGGCCAA:3
CCTGGCAA->CCTGGCTG:2
CCTGGCAA->CAAGGAAC:4
CAAGGAAC->CACAGGAT:4
CAAGGAAC->TGAGTACC:4
CAAGGAAC->CCTGGCAA:4


In [249]:
'''Solve the Nearest Neighbors of a Tree Problem.
     Input: Two internal nodes a and b specifying an edge e, followed by an adjacency list of an unrooted binary tree.
     Output: Two adjacency lists representing the nearest neighbors of the tree with respect to e. Separate the
     adjacency lists with a blank line.'''


def NearestNeighborsofTree(node0,node1,T):
    '''
    INPUT
    node0=5
    node1=4
    T={0: [4], 1: [4], 2: [5], 3: [5], 4: [0, 1, 5], 5: [2, 3, 4]}
    OUTPUT
    T2={0: [5], 1: [4], 2: [4], 3: [5], 4: [1, 5, 2], 5: [3, 4, 0]}
    T3={0: [4], 1: [5], 2: [4], 3: [5], 4: [0, 5, 2], 5: [3, 4, 1]}
    '''
    
    #### find subtrees and swap them 
    a,b=[item for item in T[node0] if item!= node1]
    c,d=[item for item in T[node1] if item!= node0]
    T2=T.copy()
    T2.update({node0:[item for item in T2[node0] if item!= a]+[c],node1:[item for item in T2[node1] if item!= c]+[a],a:[item for item in T2[a] if item!= node0]+[node1],c:[item for item in T2[c] if item!= node1]+[node0]})
    T3=T.copy()
    T3.update({node0:[item for item in T3[node0] if item!= a]+[d],node1:[item for item in T3[node1] if item!= d]+[a],a:[item for item in T3[a] if item!= node0]+[node1],d:[item for item in T3[d] if item!= node1]+[node0]})
    return T2,T3

f=open("input.txt",'r')
#f=open("input2.txt",'r')
#f=open("../../Downloads/dataset_10336_6.txt",'r')
#f=open("../../Downloads/rosalind_ba7g.txt",'r')
lines=f.read().splitlines()
f.close()
node0,node1=map(int,lines[0].split())
adj=lines[1:]

print node0,node1
#print adj

T={}
n=0
seqs={}
for item in adj:
    node=item.split('->')
    try:
        node[0]=int(node[0]) #first item is a number(4->AATGGC)
        try:
            node[1]=int(node[1]) # second item is a number(4->5)
            if node[0] in T.keys():
                T[node[0]]+=[node[1]]
            else:
                T[node[0]]=[node[1]]
                seqs[node[0]]=''
        except ValueError:    
            print 'ohoh...'
            n+=1
    except ValueError:
        pass
        
print T
#T2,T3=NearestNeighborsofTree(node0,node1,adj)
#print T2,T3

fa=open('output.txt','w')
fa.close()

for key,value in T2.items():
    for item in value:
        fa=open('output.txt','a')
        entry=str(key)+'->'+str(item)
        print entry
        fa.write(entry+'\n')
        fa.close()

fa=open('output.txt','a')
entry='\n'
print '\n'
fa.write(entry)
fa.close()
       
for key,value in T3.items():
    for item in value:
        fa=open('output.txt','a')
        entry=str(key)+'->'+str(item)
        print entry
        fa.write(entry+'\n')
        fa.close()


5 4
{0: [4], 1: [4], 2: [5], 3: [5], 4: [0, 1, 5], 5: [2, 3, 4]}
0->5
1->4
2->4
3->5
4->1
4->5
4->2
5->3
5->4
5->0


0->4
1->5
2->4
3->5
4->0
4->5
4->2
5->3
5->4
5->1


In [270]:
'''
NearestNeighborInterchange(Strings)
     score ← ∞
     generate an arbitrary unrooted binary tree Tree with |Strings| leaves
     label the leaves of Tree by arbitrary strings from Strings
     solve  the  Small Parsimony in an Unrooted Tree Problem for Tree
     label the internal nodes of Tree according to a most parsimonious labeling
     newScore ← the parsimony score of Tree
     newTree ← Tree
     while newScore < score
          score ← newScore
          Tree ← newTree
          for each internal edge e in Tree
               for each nearest neighbor NeighborTree of Tree with respect to the edge e
                    solve the Small Parsimony in an Unrooted Tree Problem for NeighborTree
                    neighborScore ← the minimum parsimony score of NeighborTree
                    if neighborScore < newScore
                         newScore ← neighborScore
                         newTree ← NeighborTree
     return newTree
'''


### See copy

'\nNearestNeighborInterchange(Strings)\n     score \xe2\x86\x90 \xe2\x88\x9e\n     generate an arbitrary unrooted binary tree Tree with |Strings| leaves\n     label the leaves of Tree by arbitrary strings from Strings\n     solve  the  Small Parsimony in an Unrooted Tree Problem for Tree\n     label the internal nodes of Tree according to a most parsimonious labeling\n     newScore \xe2\x86\x90 the parsimony score of Tree\n     newTree \xe2\x86\x90 Tree\n     while newScore < score\n          score \xe2\x86\x90 newScore\n          Tree \xe2\x86\x90 newTree\n          for each internal edge e in Tree\n               for each nearest neighbor NeighborTree of Tree with respect to the edge e\n                    solve the Small Parsimony in an Unrooted Tree Problem for NeighborTree\n                    neighborScore \xe2\x86\x90 the minimum parsimony score of NeighborTree\n                    if neighborScore < newScore\n                         newScore \xe2\x86\x90 neighborScore\n       

In [1]:
from Bioinf_functions import delta